# 1. Import libraries

In [99]:
import numpy as np
import pandas as pd

from math import sqrt, cos

import networkx as nx

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Tahoma' # change font family for Thai characters

import datetime
from datetime import time, date, datetime

# 2. Read raw data, drop unused column

In [100]:
raw_df = pd.read_excel('Capstone SampleData 14days.xlsx', sheet_name='Sheet1')

In [101]:
raw_df.head(3)

,JobAcceptedDate,jobAcceptedTime,calledMerchantTime,arrivedAtMerchantTime,mealPickedUpTime,arrivedAtCustLocationTime,foodDeliveredTime,riderInitial.lat,riderInitial.long,Merchant.Lat,Merchant.Lng,Customer.lat,Customer.lng,MerchantName,NationFoodCategory,FoodCategories
0,2020-10-03,20:39:15.840000,20:40:51.840000,20:48:19.840000,20:49:18.840000,21:14:09.840000,21:15:12.840000,13.825853,100.551958,13.825703,100.559309,13.756065,100.562375,3K SEA ZABB,Thai,อาหารทะเล
1,2020-10-01,12:54:34.560000,12:55:38.560000,13:10:26.560000,13:24:56.560000,14:01:00.560000,14:01:39.560000,13.819283,100.561874,13.831056,100.570731,13.774685,100.573913,ร้านอาหารอีสานแม่อลิส,Thai,อาหารอีสาน
2,2020-10-01,12:32:06.720000,12:32:21.720000,12:41:08.720000,12:46:41.720000,13:22:01.720000,13:22:53.720000,13.827041,100.554284,13.827205,100.536048,13.743898,100.571659,ไก่ย่างห้าดาว กรุงเทพ-นนท์44,Thai,Fast Food


In [102]:
raw_df.shape

(10000, 16)

In [103]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   JobAcceptedDate            10000 non-null  datetime64[ns]
 1   jobAcceptedTime            10000 non-null  object        
 2   calledMerchantTime         10000 non-null  object        
 3   arrivedAtMerchantTime      10000 non-null  object        
 4   mealPickedUpTime           10000 non-null  object        
 5   arrivedAtCustLocationTime  10000 non-null  object        
 6   foodDeliveredTime          10000 non-null  object        
 7   riderInitial.lat           10000 non-null  float64       
 8   riderInitial.long          10000 non-null  float64       
 9   Merchant.Lat               10000 non-null  float64       
 10  Merchant.Lng               10000 non-null  float64       
 11  Customer.lat               10000 non-null  float64       
 12  Custo

In [104]:
# We define delivery time 
# Remove the columns that are not needed
drop_list = ['jobAcceptedTime',
             'arrivedAtMerchantTime',
             'arrivedAtCustLocationTime',
             'riderInitial.lat',
             'riderInitial.long',
             'MerchantName',
             'Customer.lat',
             'Customer.lng',
             'foodDeliveredTime']
raw_df.drop(drop_list, axis=1, inplace=True)

raw_df.head(3)

,JobAcceptedDate,calledMerchantTime,mealPickedUpTime,Merchant.Lat,Merchant.Lng,NationFoodCategory,FoodCategories
0,2020-10-03,20:40:51.840000,20:49:18.840000,13.825703,100.559309,Thai,อาหารทะเล
1,2020-10-01,12:55:38.560000,13:24:56.560000,13.831056,100.570731,Thai,อาหารอีสาน
2,2020-10-01,12:32:21.720000,12:46:41.720000,13.827205,100.536048,Thai,Fast Food


# 3. Extract features

## 3.1 NationFoodCategory and FoodCategories

In [105]:
nominal_columns = ["NationFoodCategory", "FoodCategories"]
dummy_df = pd.get_dummies(raw_df[nominal_columns], drop_first=False)
raw_df = pd.concat([raw_df, dummy_df], axis=1)

In [106]:
NationFoodCategory_map = { feat:idx for idx,feat in enumerate(raw_df["NationFoodCategory"].unique())}
NationFoodCategory_mapinv = { idx:feat for idx,feat in enumerate(raw_df["NationFoodCategory"].unique())}

raw_df["NationFoodCategory_id"] = raw_df["NationFoodCategory"].map(NationFoodCategory_map)

FoodCategories_map = { feat:idx for idx,feat in enumerate(raw_df["FoodCategories"].unique())}
FoodCategories_mapinv = { idx:feat for idx,feat in enumerate(raw_df["FoodCategories"].unique())}

raw_df["FoodCategories_id"] = raw_df["FoodCategories"].map(FoodCategories_map)

In [107]:

# TODO: Angular distance encode
raw_df["NationFoodCategory_sin"] = np.sin(raw_df["NationFoodCategory_id"]*(2.*np.pi/len(raw_df["NationFoodCategory_id"].unique())))
raw_df["NationFoodCategory_cos"] = np.cos(raw_df["NationFoodCategory_id"]*(2.*np.pi/len(raw_df["NationFoodCategory_id"].unique())))

raw_df["FoodCategories_sin"] = np.sin(raw_df["FoodCategories_id"]*(2.*np.pi/len(raw_df["FoodCategories_id"].unique())))
raw_df["FoodCategories_cos"] = np.cos(raw_df["FoodCategories_id"]*(2.*np.pi/len(raw_df["FoodCategories_id"].unique())))

## 3.2 Euclidian and Shortest distance

In [108]:
# We can retrieve euc distance from food delivery data
tmp_df = pd.read_csv('Sample 14days_food_delivery_All.csv')
drop_list = ['foodDeliveredTime',
             'Customer.lat',
             'Customer.lng',
             'Merchant_to_Customer_EucDistance',
             'Merchant_to_Customer_ShortestDistance',
             'mealPickedUpTime_to_arrivedAtCustLocationTime',
             'mealPickedUpTime_to_arrivedAtCustLocationTime (s)',
             'mealPickedUpTime_to_arrivedAtCustLocationTime (m)']
tmp_df.drop(drop_list, axis=1, inplace=True)

In [109]:
raw_df['riderInitial_to_Merchant_EucDistance'] = tmp_df['riderInitial_to_Merchant_EucDistance']
raw_df['riderInitial_to_Merchant_ShortestDistance'] = tmp_df['riderInitial_to_Merchant_ShortestDistance']

## 3.3 day_of_week

In [110]:
raw_df['day_of_week'] = tmp_df['day_of_week']
raw_df['day_of_week_name'] = tmp_df['day_of_week_name']
raw_df['day_of_week_sin'] = tmp_df['day_of_week_sin']
raw_df['day_of_week_cos'] = tmp_df['day_of_week_cos']

## 3.4 isHoliday

In [111]:
raw_df['isHoliday'] = tmp_df['isHoliday']

## 3.5 calledMerchantTime_to_arrivedAtMerchantTime duration prediction data

In [112]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import pickle as pkl
model_s = pkl.load(open('food_delivery_CV_gbdt_s.pkl', 'rb')) 
model_m = pkl.load(open('food_delivery_CV_gbdt_m.pkl', 'rb')) 
food_del_df = pd.read_csv('Sample 14days_food_delivery_Cleaned.csv')
X = food_del_df.iloc[:10000, :-2]
pred_s = model_s.best_estimator_.predict(X)
pred_m = model_m.best_estimator_.predict(X)

c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- FoodCategories_cos
- FoodCategories_sin
- Merchant_lat
- Merchant_lon
- NationFoodCategory_cos
- ...
Feature names seen at fit time, yet now missing:
- euc_dist
- shortest_dist
- u_lat
- u_lon
- v_lat
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 12 features, but GradientBoostingRegressor is expecting 8 features as input.

In [ ]:
raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'] = pred_s
raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'] = pred_m

In [ ]:
raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'].describe()

count    10000.000000
mean       593.294095
std        201.997433
min        401.262823
25%        401.262823
50%        633.470806
75%        636.446666
max       1318.063090
Name: calledMerchantTime_to_arrivedAtMerchantTime_prediction (s), dtype: float64

In [ ]:
raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'].describe()

count    10000.000000
mean         9.888235
std          3.366624
min          6.687714
25%          6.687714
50%         10.557847
75%         10.607444
max         21.967718
Name: calledMerchantTime_to_arrivedAtMerchantTime_prediction (m), dtype: float64

## 3.6 datetime

In [ ]:
def to_datetime(df, date_col, time_col):
    return pd.to_datetime(df[date_col].astype(str) + " " + df[time_col].astype(str))

In [ ]:
to_datetime_feats = ['calledMerchantTime', 'mealPickedUpTime']
for feat in to_datetime_feats:
    raw_df[feat] = to_datetime(raw_df, 'JobAcceptedDate', feat)

## 3.7 duration

In [ ]:
def to_duration(df, start_col, end_col):
    return df[end_col] - df[start_col]

In [ ]:
duration_feats = [('calledMerchantTime', 'mealPickedUpTime')]

for feat_pairs in duration_feats:
    raw_df[feat_pairs[0] + '_to_' + feat_pairs[1]] = to_duration(raw_df, feat_pairs[0], feat_pairs[1])
    raw_df[feat_pairs[0] + '_to_' + feat_pairs[1] + ' (s)'] = raw_df[feat_pairs[0] + '_to_' + feat_pairs[1]].dt.total_seconds()
    raw_df[feat_pairs[0] + '_to_' + feat_pairs[1] + ' (m)'] = raw_df[feat_pairs[0] + '_to_' + feat_pairs[1]].dt.total_seconds() / 60

In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 53 columns):
 #   Column                                                      Non-Null Count  Dtype          
---  ------                                                      --------------  -----          
 0   JobAcceptedDate                                             10000 non-null  datetime64[ns] 
 1   calledMerchantTime                                          10000 non-null  datetime64[ns] 
 2   mealPickedUpTime                                            10000 non-null  datetime64[ns] 
 3   Merchant.Lat                                                10000 non-null  float64        
 4   Merchant.Lng                                                10000 non-null  float64        
 5   NationFoodCategory                                          10000 non-null  object         
 6   FoodCategories                                              10000 non-null  object         
 7   NationFoodCate

In [ ]:
raw_df.to_csv('Sample 14days_food_preparation_All.csv', index=False)

# 4. Prepare Data for ML

## 4.1 load extracted feature data

In [ ]:
raw_df = pd.read_csv('Sample 14days_food_preparation_All.csv')

In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 53 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   JobAcceptedDate                                             10000 non-null  object 
 1   calledMerchantTime                                          10000 non-null  object 
 2   mealPickedUpTime                                            10000 non-null  object 
 3   Merchant.Lat                                                10000 non-null  float64
 4   Merchant.Lng                                                10000 non-null  float64
 5   NationFoodCategory                                          10000 non-null  object 
 6   FoodCategories                                              10000 non-null  object 
 7   NationFoodCategory_International                            10000 non-null  int64  
 8

## 4.2 construct new dataframe

In [ ]:

# TODO: data_dict_1 contains one-hot encoding for NationFoodCategory and FoodCategories
data_dict_1 = {
   'Merchant_lat':[],
   'Merchant_lon':[],
   'NationFoodCategory_International':[],
   'NationFoodCategory_Isram':[],
   'NationFoodCategory_Japanese':[],
   'NationFoodCategory_Korean':[],
   'NationFoodCategory_Myanmar':[],          
   'NationFoodCategory_Thai':[],
   'NationFoodCategory_Vietnam':[],
   'FoodCategories_FastFood':[],
   'FoodCategories_QuickMeal':[],
   'FoodCategories_ขนมจีน':[],
   'FoodCategories_ของหวาน':[],
   'FoodCategories_ปิ้งย่าง':[],
   'FoodCategories_พิซซ่า':[],          
   'FoodCategories_ร้านก๋วยเตี๋ยว':[],          
   'FoodCategories_ร้านอาหาร':[],          
   'FoodCategories_สปาเก็ตตี้':[],          
   'FoodCategories_สุกี้ยากี้':[],
   'FoodCategories_สเต๊ก':[],
   'FoodCategories_อาหารคลีน':[],
   'FoodCategories_อาหารจานด่วน':[],
   'FoodCategories_อาหารตามสั่ง':[],
   'FoodCategories_อาหารทะเล':[],
   'FoodCategories_อาหารอีสาน':[],
   'FoodCategories_อาหารฮาลาล':[],
   'FoodCategories_อาหารเหนือ':[],
   'FoodCategories_อาหารใต้':[],
   'FoodCategories_เครื่องดื่ม':[],
   'FoodCategories_ไก่ทอด':[],
   'riderInitial_to_Merchant_EucDistance':[],
   'riderInitial_to_Merchant_ShortestDistance':[],
   'day_of_week_sin':[],
   'day_of_week_cos':[],
   'calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)':[],
   'calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)':[],
   'duration (s)':[],
   'duration (m)':[]
}

# TODO: data_dict_2 contains angular distance encoding for NationFoodCategory and FoodCategories
data_dict_2 = {
   'Merchant_lat':[],
   'Merchant_lon':[],
   'NationFoodCategory_sin':[],
   'NationFoodCategory_cos':[],
   'FoodCategories_sin':[],
   'FoodCategories_cos':[],
   'riderInitial_to_Merchant_EucDistance':[],
   'riderInitial_to_Merchant_ShortestDistance':[],
   'day_of_week_sin':[],
   'day_of_week_cos':[],
   'calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)':[],
   'calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)':[],
   'duration (s)':[],
   'duration (m)':[]
}

## 4.3 fill data from raw_df to data_dict

In [ ]:
data_dict_1['Merchant_lat'] += raw_df['Merchant.Lat'].tolist()
data_dict_1['Merchant_lon'] += raw_df['Merchant.Lng'].tolist()
data_dict_1['NationFoodCategory_International'] += raw_df['NationFoodCategory_International'].tolist()
data_dict_1['NationFoodCategory_Isram'] += raw_df['NationFoodCategory_Isram'].tolist()
data_dict_1['NationFoodCategory_Japanese'] += raw_df['NationFoodCategory_Japanese'].tolist()
data_dict_1['NationFoodCategory_Korean'] += raw_df['NationFoodCategory_Korean'].tolist()
data_dict_1['NationFoodCategory_Myanmar'] += raw_df['NationFoodCategory_Myanmar'].tolist()
data_dict_1['NationFoodCategory_Thai'] += raw_df['NationFoodCategory_Thai'].tolist()
data_dict_1['NationFoodCategory_Vietnam'] += raw_df['NationFoodCategory_Vietnam'].tolist()
data_dict_1['FoodCategories_FastFood'] += raw_df['FoodCategories_Fast Food'].tolist()
data_dict_1['FoodCategories_QuickMeal'] += raw_df['FoodCategories_Quick Meal'].tolist()
data_dict_1['FoodCategories_ขนมจีน'] += raw_df['FoodCategories_ขนมจีน'].tolist()
data_dict_1['FoodCategories_ของหวาน'] += raw_df['FoodCategories_ของหวาน'].tolist()
data_dict_1['FoodCategories_ปิ้งย่าง'] += raw_df['FoodCategories_ปิ้งย่าง'].tolist()
data_dict_1['FoodCategories_พิซซ่า'] += raw_df['FoodCategories_พิซซ่า'].tolist()
data_dict_1['FoodCategories_ร้านก๋วยเตี๋ยว'] += raw_df['FoodCategories_ร้านก๋วยเตี๋ยว'].tolist()
data_dict_1['FoodCategories_ร้านอาหาร'] += raw_df['FoodCategories_ร้านอาหาร'].tolist()
data_dict_1['FoodCategories_สปาเก็ตตี้'] += raw_df['FoodCategories_สปาเก็ตตี้'].tolist()
data_dict_1['FoodCategories_สุกี้ยากี้'] += raw_df['FoodCategories_สุกี้ยากี้'].tolist()
data_dict_1['FoodCategories_สเต๊ก'] += raw_df['FoodCategories_สเต๊ก'].tolist()
data_dict_1['FoodCategories_อาหารคลีน'] += raw_df['FoodCategories_อาหารคลีน'].tolist()
data_dict_1['FoodCategories_อาหารจานด่วน'] += raw_df['FoodCategories_อาหารจานด่วน'].tolist()
data_dict_1['FoodCategories_อาหารตามสั่ง'] += raw_df['FoodCategories_อาหารตามสั่ง'].tolist()
data_dict_1['FoodCategories_อาหารทะเล'] += raw_df['FoodCategories_อาหารทะเล'].tolist()
data_dict_1['FoodCategories_อาหารอีสาน'] += raw_df['FoodCategories_อาหารอีสาน'].tolist()
data_dict_1['FoodCategories_อาหารฮาลาล'] += raw_df['FoodCategories_อาหารฮาลาล'].tolist()
data_dict_1['FoodCategories_อาหารเหนือ'] += raw_df['FoodCategories_อาหารเหนือ'].tolist()
data_dict_1['FoodCategories_อาหารใต้'] += raw_df['FoodCategories_อาหารใต้'].tolist()
data_dict_1['FoodCategories_เครื่องดื่ม'] += raw_df['FoodCategories_เครื่องดื่ม'].tolist()
data_dict_1['FoodCategories_ไก่ทอด'] += raw_df['FoodCategories_ไก่ทอด'].tolist()
data_dict_1['riderInitial_to_Merchant_EucDistance'] += raw_df['riderInitial_to_Merchant_EucDistance'].tolist()
data_dict_1['riderInitial_to_Merchant_ShortestDistance'] += raw_df['riderInitial_to_Merchant_ShortestDistance'].tolist()
data_dict_1['day_of_week_sin'] += raw_df['day_of_week_sin'].tolist()
data_dict_1['day_of_week_cos'] += raw_df['day_of_week_cos'].tolist()
data_dict_1['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'] += raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'].tolist()
data_dict_1['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'] += raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'].tolist()
data_dict_1['duration (s)'] += raw_df['calledMerchantTime_to_mealPickedUpTime (s)'].tolist()
data_dict_1['duration (m)'] += raw_df['calledMerchantTime_to_mealPickedUpTime (m)'].tolist()

In [ ]:
data_dict_2['Merchant_lat'] += raw_df['Merchant.Lat'].tolist()
data_dict_2['Merchant_lon'] += raw_df['Merchant.Lng'].tolist()
data_dict_2['NationFoodCategory_sin'] += raw_df['NationFoodCategory_sin'].tolist()
data_dict_2['NationFoodCategory_cos'] += raw_df['NationFoodCategory_cos'].tolist()
data_dict_2['FoodCategories_sin'] += raw_df['FoodCategories_sin'].tolist()
data_dict_2['FoodCategories_cos'] += raw_df['FoodCategories_cos'].tolist()
data_dict_2['riderInitial_to_Merchant_EucDistance'] += raw_df['riderInitial_to_Merchant_EucDistance'].tolist()
data_dict_2['riderInitial_to_Merchant_ShortestDistance'] += raw_df['riderInitial_to_Merchant_ShortestDistance'].tolist()
data_dict_2['day_of_week_sin'] += raw_df['day_of_week_sin'].tolist()
data_dict_2['day_of_week_cos'] += raw_df['day_of_week_cos'].tolist()
data_dict_2['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'] += raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (s)'].tolist()
data_dict_2['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'] += raw_df['calledMerchantTime_to_arrivedAtMerchantTime_prediction (m)'].tolist()
data_dict_2['duration (s)'] += raw_df['calledMerchantTime_to_mealPickedUpTime (s)'].tolist()
data_dict_2['duration (m)'] += raw_df['calledMerchantTime_to_mealPickedUpTime (m)'].tolist()

In [ ]:
df_1 = pd.DataFrame(data_dict_1)
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 38 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Merchant_lat                                                10000 non-null  float64
 1   Merchant_lon                                                10000 non-null  float64
 2   NationFoodCategory_International                            10000 non-null  int64  
 3   NationFoodCategory_Isram                                    10000 non-null  int64  
 4   NationFoodCategory_Japanese                                 10000 non-null  int64  
 5   NationFoodCategory_Korean                                   10000 non-null  int64  
 6   NationFoodCategory_Myanmar                                  10000 non-null  int64  
 7   NationFoodCategory_Thai                                     10000 non-null  int64  
 8

In [ ]:
df_2 = pd.DataFrame(data_dict_2)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Merchant_lat                                                10000 non-null  float64
 1   Merchant_lon                                                10000 non-null  float64
 2   NationFoodCategory_sin                                      10000 non-null  float64
 3   NationFoodCategory_cos                                      10000 non-null  float64
 4   FoodCategories_sin                                          10000 non-null  float64
 5   FoodCategories_cos                                          10000 non-null  float64
 6   riderInitial_to_Merchant_EucDistance                        10000 non-null  float64
 7   riderInitial_to_Merchant_ShortestDistance                   10000 non-null  float64
 8

## 4.4 save data

In [113]:
df_1.to_csv('Sample 14days_food_preparation_Cleaned_1.csv', index=False)
df_2.to_csv('Sample 14days_food_preparation_Cleaned_2.csv', index=False)
df_2.to_csv('Sample 14days_food_preparation_Cleaned.csv', index=False)